In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
# Risk weights based on Basel III standards
def assign_basel_risk_weight(pd_value):
    # Simplified risk weight assignment based on Basel III
    if pd_value <= 0.05:
        return 0.5  # 50% risk weight
    elif pd_value <= 0.10:
        return 0.75  # 75% risk weight
    elif pd_value <= 0.30:
        return 1.0  # 100% risk weight
    else:
        return 1.5  # 150% risk weight

In [3]:
# Load Basel risk calculations
loans_df_test = pd.read_csv('../data/processed/basel_risk_calculations.csv')

In [4]:
# Create stress scenario functions
def apply_stress_scenario(base_pd, scenario='moderate'):
    """Apply stress multipliers to PD values"""
    scenarios = {
        'mild': 1.5,       # 50% increase in defaults
        'moderate': 2.0,   # Double defaults
        'severe': 3.0      # Triple defaults
    }
    
    multiplier = scenarios.get(scenario, 1.0)
    stressed_pd = base_pd * multiplier
    
    # Cap at 1.0
    return np.minimum(stressed_pd, 1.0)


In [9]:
# Apply moderate stress scenario
loans_df_test['Stressed_PD'] = apply_stress_scenario(loans_df_test['PD'], scenario='moderate')
loans_df_test['Stressed_EL'] = loans_df_test['Stressed_PD'] * loans_df_test['LGD'] * loans_df_test['EAD']
loans_df_test['Stressed_RiskWeight'] = loans_df_test['Stressed_PD'].apply(lambda x: assign_basel_risk_weight(x))
loans_df_test['Stressed_RWA'] = loans_df_test['EAD'] * loans_df_test['Stressed_RiskWeight']

In [10]:
# Portfolio level calculations
total_exposure = loans_df_test['EAD'].sum()
total_rwa = loans_df_test['RWA'].sum()
total_stressed_rwa = loans_df_test['Stressed_RWA'].sum()

In [11]:
# Basel III minimum capital requirements
tier1_capital_ratio = 0.06  # 6%
total_capital_ratio = 0.08  # 8%
conservation_buffer = 0.025  # 2.5%

In [12]:
# Calculate minimum capital requirements
min_tier1_capital = total_rwa * tier1_capital_ratio
min_total_capital = total_rwa * total_capital_ratio
min_capital_with_buffer = total_rwa * (total_capital_ratio + conservation_buffer)

In [13]:
# Calculate stressed capital requirements
stressed_min_tier1_capital = total_stressed_rwa * tier1_capital_ratio
stressed_min_total_capital = total_stressed_rwa * total_capital_ratio
stressed_min_capital_with_buffer = total_stressed_rwa * (total_capital_ratio + conservation_buffer)


In [14]:
# Display results
print(f"Total Exposure: ${total_exposure:,.2f}")
print(f"Total Risk-Weighted Assets: ${total_rwa:,.2f}")
print(f"Minimum Tier 1 Capital Required: ${min_tier1_capital:,.2f}")
print(f"Minimum Total Capital Required: ${min_total_capital:,.2f}")
print(f"Minimum Capital with Conservation Buffer: ${min_capital_with_buffer:,.2f}")
print("\nStressed Scenario:")
print(f"Total Stressed Risk-Weighted Assets: ${total_stressed_rwa:,.2f}")
print(f"Stressed Capital Required: ${stressed_min_total_capital:,.2f}")

Total Exposure: $6,916,273,475.00
Total Risk-Weighted Assets: $6,056,394,025.00
Minimum Tier 1 Capital Required: $363,383,641.50
Minimum Total Capital Required: $484,511,522.00
Minimum Capital with Conservation Buffer: $635,921,372.63

Stressed Scenario:
Total Stressed Risk-Weighted Assets: $7,727,702,381.25
Stressed Capital Required: $618,216,190.50


In [15]:
# Save stress test results
stress_results = pd.DataFrame({
    'Metric': ['Total Exposure', 'RWA', 'Tier 1 Capital', 'Total Capital', 'With Buffer'],
    'Normal': [total_exposure, total_rwa, min_tier1_capital, 
              min_total_capital, min_capital_with_buffer],
    'Stressed': [total_exposure, total_stressed_rwa, stressed_min_tier1_capital,
                stressed_min_total_capital, stressed_min_capital_with_buffer]
})

In [16]:
stress_results.to_csv('../data/processed/stress_test_results.csv', index=False)